## 作業
礙於不是所有同學都有 GPU ，這邊的範例使用的是簡化版本的 ResNet，確保所有同學都能夠順利訓練!


最後一天的作業請閱讀這篇非常詳盡的[文章](https://blog.gtwang.org/programming/keras-resnet-50-pre-trained-model-build-dogs-cats-image-classification-system/)，基本上已經涵蓋了所有訓練　CNN 常用的技巧，請使用所有學過的訓練技巧，盡可能地提高 Cifar-10 的 test data 準確率，截圖你最佳的結果並上傳來完成最後一次的作業吧!

另外這些技巧在 Kaggle 上也會被許多人使用，更有人會開發一些新的技巧，例如使把預訓練在 ImageNet 上的模型當成 feature extractor 後，再拿擷取出的特徵重新訓練新的模型，這些技巧再進階的課程我們會在提到，有興趣的同學也可以[參考](https://www.kaggle.com/insaff/img-feature-extraction-with-pretrained-resnet)

In [1]:
# Loading library
from keras.datasets import cifar10
from keras.models import Sequential
from keras.layers import Dense, Dropout, Activation, Flatten
from keras.layers import Conv2D, MaxPooling2D
from keras.optimizers import RMSprop, Adam

Using TensorFlow backend.


In [2]:
(x_train, y_train), (x_test, y_test) = cifar10.load_data()
print("x_train shape: ", x_train.shape)
print("x_test shape: ", x_test.shape)

x_train shape:  (50000, 32, 32, 3)
x_test shape:  (10000, 32, 32, 3)


總共有 50,000 筆 train data, 每筆 data 的大小為 32 * 32 * 3。  
總共有 10,000 筆 train data, 每筆 data 的大小為 32 * 32 * 3。

In [3]:
# Transformation.
x_train = x_train.astype('float32')
x_test = x_test.astype('float32')

# normalization
x_train = x_train / 255.
x_test = x_test / 255.

from keras.utils import to_categorical

y_train = to_categorical(y_train, 10)
y_test = to_categorical(y_test, 10)

In [6]:
class Model():
    def __init__(self, x_train, y_train, x_test, y_test):
        self.FirConvSize = None
        self.SecConvSize = None
        self.Padding = None
        self.model = None
        self.KernlSize = None
        self.x_train = x_train
        self.y_train = y_train
        self.x_test = x_test
        self.y_test = y_test
    
    def setHypParam(self, padding='same', KernlSize=(3, 3), FirConvSize=32, SecConvSize=64):
        self.FirConvSize = FirConvSize
        self.SecConvSize = SecConvSize
        self.KernlSize = KernlSize
        self.Padding = padding
    
    def buildModel(self):
        self.model = Sequential()
        self.model.add(Conv2D(self.FirConvSize, self.KernlSize, padding=self.Padding,
                         input_shape=x_train.shape[1:]))
        self.model.add(Activation('relu'))
        self.model.add(Conv2D(self.FirConvSize, self.KernlSize))
        self.model.add(Activation('relu'))
        self.model.add(MaxPooling2D(pool_size=(2, 2)))
        self.model.add(Dropout(0.25))
        
        self.model.add(Conv2D(self.SecConvSize, self.KernlSize, padding=self.Padding))
        self.model.add(Activation('relu'))
        self.model.add(Conv2D(self.SecConvSize, self.KernlSize))
        self.model.add(Activation('relu'))
        self.model.add(MaxPooling2D(pool_size=(2, 2)))
        self.model.add(Dropout(0.25))

        self.model.add(Flatten())
        self.model.add(Dense(512))
        self.model.add(Activation('relu'))
        self.model.add(Dropout(0.5))
        self.model.add(Dense(10))
        self.model.add(Activation('softmax'))
        
        self.model.summary()
    
    def setCompile(self):
        self.model.compile(loss='categorical_crossentropy',
                           optimizer=RMSprop(),
                           metrics=['accuracy'])
    
    def fitModel(self):
        self.model.fit(self.x_train, self.y_train,
                       batch_size=128, 
                       epochs=30,
                       verbose=False,
                       validation_data=(self.x_test, self.y_test))
    
    def evaluateScore(self):
        return (self.model.evaluate(self.x_test, self.y_test, verbose=0))

In [7]:
FirConvSize = 64
SecConvSize = 64

model = Model(x_train, y_train, x_test, y_test)
model.setHypParam(FirConvSize=FirConvSize, SecConvSize=SecConvSize)
model.buildModel()
model.setCompile()
model.fitModel()
score = model.evaluateScore()
print('Test loss:%.4f' %score[0])
print('Test accuracy:%.4f' %score[1])

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_5 (Conv2D)            (None, 32, 32, 64)        1792      
_________________________________________________________________
activation_7 (Activation)    (None, 32, 32, 64)        0         
_________________________________________________________________
conv2d_6 (Conv2D)            (None, 30, 30, 64)        36928     
_________________________________________________________________
activation_8 (Activation)    (None, 30, 30, 64)        0         
_________________________________________________________________
max_pooling2d_3 (MaxPooling2 (None, 15, 15, 64)        0         
_________________________________________________________________
dropout_4 (Dropout)          (None, 15, 15, 64)        0         
_________________________________________________________________
conv2d_7 (Conv2D)            (None, 15, 15, 64)        36928     
__________